In [ ]:
# 필요한 라이브러리 임포트
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# PyTorch 임포트
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from torchvision import datasets, transforms

# 1. 데이터셋 이해 및 문제 정의
# - 데이터셋: 패션 MNIST (Fashion MNIST)
# - 목표: 주어진 의류 이미지를 10개의 카테고리 중 하나로 분류하는 모델을 구축합니다.
# - 문제 유형: 다중 클래스 분류
# - 평가 지표: 정확도(Accuracy)

# 2. 데이터 로드 및 탐색적 데이터 분석(EDA)

# 2.1 데이터 로드 및 전처리
# 데이터 변환 정의 (Tensor로 변환 및 정규화)
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

# 훈련 데이터셋 및 테스트 데이터셋 로드
train_dataset = datasets.FashionMNIST(
    root='./data',
    train=True,
    download=True,
    transform=transform
)

test_dataset = datasets.FashionMNIST(
    root='./data',
    train=False,
    download=True,
    transform=transform
)

# 데이터 로더 정의
batch_size = 64

train_loader = DataLoader(
    dataset=train_dataset,
    batch_size=batch_size,
    shuffle=True
)

test_loader = DataLoader(
    dataset=test_dataset,
    batch_size=batch_size,
    shuffle=False
)

# 클래스 이름 정의
class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
               'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

# 2.2 데이터 샘플 시각화
# 첫 번째 배치에서 이미지와 레이블 가져오기
dataiter = iter(train_loader)
images, labels = dataiter.next()

# 이미지 시각화
plt.figure()
plt.imshow(images[0].numpy().squeeze(), cmap='gray')
plt.title(f"레이블: {labels[0].item()} ({class_names[labels[0]]})")
plt.colorbar()
plt.show()

# 여러 이미지 시각화
figure = plt.figure(figsize=(8, 8))
cols, rows = 5, 5
for i in range(1, cols * rows + 1):
    img = images[i - 1].numpy().squeeze()
    label = labels[i - 1]
    figure.add_subplot(rows, cols, i)
    plt.title(class_names[label])
    plt.axis('off')
    plt.imshow(img, cmap='gray')
plt.show()

# 3. 모델 선택 및 설계

# 3.1 DNN 모델 설계
class DNN(nn.Module):
    def __init__(self):
        super(DNN, self).__init__()
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(28 * 28, 512)
        self.relu1 = nn.ReLU()
        self.dropout1 = nn.Dropout(0.2)
        self.fc2 = nn.Linear(512, 256)
        self.relu2 = nn.ReLU()
        self.dropout2 = nn.Dropout(0.2)
        self.fc3 = nn.Linear(256, 10)
        self.softmax = nn.LogSoftmax(dim=1)  # CrossEntropyLoss와 함께 사용하기 위해 LogSoftmax

    def forward(self, x):
        x = self.flatten(x)
        x = self.relu1(self.fc1(x))
        x = self.dropout1(x)
        x = self.relu2(self.fc2(x))
        x = self.dropout2(x)
        x = self.fc3(x)
        x = self.softmax(x)
        return x

# 3.2 CNN 모델 설계
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3)  # 출력 채널 32, 커널 크기 3x3
        self.pool = nn.MaxPool2d(2, 2)    # 커널 크기 2x2, 스트라이드 2
        self.conv2 = nn.Conv2d(32, 64, 3)
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(64 * 5 * 5, 64)  # 입력 차원 계산 필요
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(64, 10)
        self.softmax = nn.LogSoftmax(dim=1)  # CrossEntropyLoss와 함께 사용하기 위해 LogSoftmax

    def forward(self, x):
        x = self.pool(self.relu(self.conv1(x)))  # [batch_size, 32, 13, 13]
        x = self.pool(self.relu(self.conv2(x)))  # [batch_size, 64, 5, 5]
        x = self.flatten(x)
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        x = self.softmax(x)
        return x

# 4. 모델 학습 및 검증

# 장치 설정 (GPU 사용 가능 시 GPU 사용)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"사용 중인 장치: {device}")

# 모델 인스턴스 생성 및 장치로 이동
dnn_model = DNN().to(device)
cnn_model = CNN().to(device)

# 손실 함수 및 옵티마이저 정의
criterion = nn.NLLLoss()  # CrossEntropyLoss와 LogSoftmax 조합 가능
dnn_optimizer = optim.Adam(dnn_model.parameters(), lr=0.001)
cnn_optimizer = optim.Adam(cnn_model.parameters(), lr=0.001)

# 4.1 DNN 모델 학습
epochs = 20
dnn_train_losses = []
dnn_train_accuracies = []
dnn_val_losses = []
dnn_val_accuracies = []

for epoch in range(epochs):
    running_loss = 0.0
    correct = 0
    total = 0

    dnn_model.train()
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        # 이미지 Flatten
        images = images.view(images.shape[0], -1)
        # 옵티마이저 초기화
        dnn_optimizer.zero_grad()
        # 순전파
        outputs = dnn_model(images)
        # 손실 계산
        loss = criterion(outputs, labels)
        # 역전파
        loss.backward()
        # 가중치 업데이트
        dnn_optimizer.step()
        # 통계
        running_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    train_loss = running_loss / len(train_loader)
    train_accuracy = 100 * correct / total
    dnn_train_losses.append(train_loss)
    dnn_train_accuracies.append(train_accuracy)

    # 검증 데이터로 평가
    dnn_model.eval()
    val_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            # 이미지 Flatten
            images = images.view(images.shape[0], -1)
            outputs = dnn_model(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    val_loss = val_loss / len(test_loader)
    val_accuracy = 100 * correct / total
    dnn_val_losses.append(val_loss)
    dnn_val_accuracies.append(val_accuracy)

    print(f"DNN Epoch [{epoch+1}/{epochs}], "
          f"Train Loss: {train_loss:.4f}, Train Acc: {train_accuracy:.2f}%, "
          f"Val Loss: {val_loss:.4f}, Val Acc: {val_accuracy:.2f}%")

# 4.2 CNN 모델 학습
cnn_train_losses = []
cnn_train_accuracies = []
cnn_val_losses = []
cnn_val_accuracies = []

for epoch in range(epochs):
    running_loss = 0.0
    correct = 0
    total = 0

    cnn_model.train()
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        # 옵티마이저 초기화
        cnn_optimizer.zero_grad()
        # 순전파
        outputs = cnn_model(images)
        # 손실 계산
        loss = criterion(outputs, labels)
        # 역전파
        loss.backward()
        # 가중치 업데이트
        cnn_optimizer.step()
        # 통계
        running_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    train_loss = running_loss / len(train_loader)
    train_accuracy = 100 * correct / total
    cnn_train_losses.append(train_loss)
    cnn_train_accuracies.append(train_accuracy)

    # 검증 데이터로 평가
    cnn_model.eval()
    val_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = cnn_model(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    val_loss = val_loss / len(test_loader)
    val_accuracy = 100 * correct / total
    cnn_val_losses.append(val_loss)
    cnn_val_accuracies.append(val_accuracy)

    print(f"CNN Epoch [{epoch+1}/{epochs}], "
          f"Train Loss: {train_loss:.4f}, Train Acc: {train_accuracy:.2f}%, "
          f"Val Loss: {val_loss:.4f}, Val Acc: {val_accuracy:.2f}%")

# 5. 모델 평가 및 시각화

# DNN 모델 정확도 시각화
plt.figure()
plt.plot(dnn_train_accuracies, label='훈련 정확도')
plt.plot(dnn_val_accuracies, label='검증 정확도')
plt.title('DNN 모델 정확도')
plt.xlabel('에포크')
plt.ylabel('정확도 (%)')
plt.legend()
plt.show()

# CNN 모델 정확도 시각화
plt.figure()
plt.plot(cnn_train_accuracies, label='훈련 정확도')
plt.plot(cnn_val_accuracies, label='검증 정확도')
plt.title('CNN 모델 정확도')
plt.xlabel('에포크')
plt.ylabel('정확도 (%)')
plt.legend()
plt.show()

# 6. 모델 저장 및 배포

# 6.1 모델 저장
torch.save(dnn_model.state_dict(), 'dnn_fashion_mnist.pth')
torch.save(cnn_model.state_dict(), 'cnn_fashion_mnist.pth')

# 6.2 모델 로드
# DNN 모델 로드
loaded_dnn_model = DNN()
loaded_dnn_model.load_state_dict(torch.load('dnn_fashion_mnist.pth'))
loaded_dnn_model.to(device)

# CNN 모델 로드
loaded_cnn_model = CNN()
loaded_cnn_model.load_state_dict(torch.load('cnn_fashion_mnist.pth'))
loaded_cnn_model.to(device)

# 6.3 새로운 데이터로 예측
# 예시로 테스트 데이터의 첫 번째 이미지 사용
loaded_cnn_model.eval()
with torch.no_grad():
    new_image = test_dataset[0][0].unsqueeze(0).to(device)  # 배치 차원 추가
    output = loaded_cnn_model(new_image)
    _, predicted_label = torch.max(output.data, 1)
    predicted_label = predicted_label.item()
    actual_label = test_dataset[0][1]

print(f"예측된 레이블: {predicted_label} ({class_names[predicted_label]})")
print(f"실제 레이블: {actual_label} ({class_names[actual_label]})")

# 이미지 시각화
plt.figure()
plt.imshow(new_image.cpu().squeeze(), cmap='gray')
plt.title(f"실제 레이블: {class_names[actual_label]}, 예측된 레이블: {class_names[predicted_label]}")
plt.axis('off')
plt.show()